In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.utils.data import DataLoader

In [5]:
# 1 Data Transformer
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,),(0.5,))])

# 2 Create Train Dataset
trainset = torchvision.datasets.MNIST(root='./data', train =True,
                                    download = True, transform = transform)
trainloader = DataLoader(trainset, batch_size =64, shuffle =True)

#3 Create Test Dataset
testset = torchvision.datasets.MNIST(root = "./data", train = False,
                                     download = True, transform = transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
class Network(nn.Module):

    def __init__(self):
        super(Network,self).__init__()
        self.fc1 = nn.Linear(28*28,1028)
        self.fc2 = nn.Linear(1028,256)
        self.fc3 = nn.Linear(256,10)
    def forward(self, x):
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
         

In [10]:
model = Network().to(device)
optimizer = optim.Adam(model.parameters(), lr =0.001)
criterion  = nn.CrossEntropyLoss().to(device)



In [12]:
def test_model(model, testloader, criterion, device):
    model.eval()  # Modeli değerlendirme moduna al
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Grad hesaplamalarını kapatıyoruz çünkü test sırasında geri yayılım yapılmaz
    with torch.no_grad():
        for data in testloader:
            X, y = data
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = criterion(y_pred, y)
            
            test_loss += loss.item()

            # Doğruluğu hesapla
            _, predicted = torch.max(y_pred, 1)
            correct_predictions += (predicted == y).sum().item()
            total_predictions += y.size(0)

    # Ortalama kayıp ve doğruluğu hesapla
    average_test_loss = test_loss / len(testloader)
    test_accuracy = correct_predictions / total_predictions

    print(f'Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')
    
    return average_test_loss, test_accuracy

In [11]:
for epoch in range(3):
    epoch_loss =0.0
    correct_predictions = 0
    total_predictions = 0

    #set to train mode
    model.train()

    #train for all batches of data
    for data in trainloader:
        X, y = data
        X , y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
                
        epoch_loss += loss.item()

        #Calculate accuracy
        _, predicted = torch.max(y_pred,1)
        correct_predictions += (predicted ==y).sum().item()
        total_predictions += y.size(0)

    accuracy = correct_predictions / total_predictions
    average_loss = epoch_loss / len(trainloader)

    print(f'Epoch {epoch+1}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

    


Epoch 1, Loss: 0.2847, Accuracy: 0.9117
Epoch 2, Loss: 0.1305, Accuracy: 0.9597
Epoch 3, Loss: 0.1004, Accuracy: 0.9689


In [16]:
torch.save(model.state_dict(),"model.pth")

In [32]:
# Model nesnesini oluştur
pruning_model = Network()

# Ağırlıkları yükle
pruning_model.load_state_dict(torch.load("model.pth", map_location=device))

# Modeli değerlendirme moduna al
pruning_model.eval()

C:\Users\user\AppData\Local\Temp\ipykernel_21908\4016724546.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pruning_model.load_state_dict(torch.load("model.pth", map_loc

Network(
  (fc1): Linear(in_features=784, out_features=1028, bias=True)
  (fc2): Linear(in_features=1028, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [36]:
test_model(model.to(device), testloader, criterion, device)

Test Loss: 0.0990, Test Accuracy: 0.9698


(0.09899375607870567, 0.9698)

In [34]:
#Define a prunin threshold (lambda)
threshold = 0.02

# Implement zero-pruning
for name, param in pruning_model.named_parameters():
    if "weight" in name:
        param.data[param.data.abs()<threshold] =0.0
        

In [35]:
test_model(pruning_model.to(device), testloader, criterion, device)

Test Loss: 0.0939, Test Accuracy: 0.9716


(0.09387334367960312, 0.9716)

In [44]:
def get_zero_percentage(model):
    total_params = 0
    zero_params = 0
    
    # Modelin state_dict'ini döngü ile dolaş
    for name, param in model.state_dict().items():
        # Sadece ağırlıkları (weights) kontrol et
        if 'weight' in name:
            total_params += param.numel()  # Toplam parametre sayısını artır
            zero_params += (param == 0).sum().item()  # Sıfır olan parametre sayısını artır
    
    # Sıfır olan ağırlıkların yüzdesini hesapla
    zero_percentage = (zero_params / total_params) * 100 if total_params > 0 else 0
    
    print(f'Toplam Weight Parametre Sayısı: {total_params}')
    print(f'Sıfır Olan Weight Parametre Sayısı: {zero_params}')
    print(f'Sıfır Olan Weight Parametre Yüzdesi: {zero_percentage:.2f}%')

    return zero_percentage

In [45]:
get_zero_percentage(model)

Toplam Weight Parametre Sayısı: 1071680
Sıfır Olan Weight Parametre Sayısı: 0
Sıfır Olan Weight Parametre Yüzdesi: 0.00%


0.0

In [46]:
get_zero_percentage(pruning_model)

Toplam Weight Parametre Sayısı: 1071680
Sıfır Olan Weight Parametre Sayısı: 528283
Sıfır Olan Weight Parametre Yüzdesi: 49.29%


49.29484547626157

In [47]:
torch.save(pruning_model.state_dict(),"pruning_model.pth")

In [49]:
#use scipy to sparse matrix
import scipy.sparse as sp

sparse_weights = []

#Convert the pruned weights to a sparse matrix
for name,param in pruning_model.named_parameters():
    if "weight" in name:

        np_weight = param.data.cpu().numpy()
        sparse_weights.append(sp.csr_matrix(np_weight))

In [50]:
sparse_weights

[<Compressed Sparse Row sparse matrix of dtype 'float32'
 	with 416541 stored elements and shape (1028, 784)>,
 <Compressed Sparse Row sparse matrix of dtype 'float32'
 	with 124963 stored elements and shape (256, 1028)>,
 <Compressed Sparse Row sparse matrix of dtype 'float32'
 	with 1893 stored elements and shape (10, 256)>]

In [55]:
total_size = 0

for name, param in pruning_model.named_parameters():

    if "weight" in name:

        tensor = param.data
        total_size += tensor.element_size() * tensor.numel()

tensor_size_mb = total_size/(1024**2)
print(tensor_size_mb)




4.088134765625


In [57]:
total_size = 0
for w in sparse_weights:
    total_size +=w.data.nbytes

#Convert to MBs
csr_size_mb = total_size /(1024**2)
print(csr_size_mb)


2.072895050048828


In [82]:
import torch
import numpy as np
from scipy.sparse import csr_matrix, save_npz

def save_model_as_csr(model, file_path):
    # Modelin parametrelerini al
    state_dict = model.state_dict()
    
    # Her bir parametreyi CSR formatında sakla
    csr_weights = {}

    for name, param in state_dict.items():
        if param.requires_grad:
            # Parametreyi numpy dizisine çevir
            dense_param = param.detach().numpy()
            # CSR matrisine dönüştür
            csr_param = csr_matrix(dense_param)
            csr_weights[name] = csr_param

    # CSR formatındaki parametreleri kaydet
    for name, csr_param in csr_weights.items():
        print
        save_npz(f"{file_path}_{name}.npz", csr_param)

    print(f"Model parameters saved as CSR format at: {file_path}")

# Örnek kullanım
save_model_as_csr(pruning_model, "pruned_model_csr")

Model parameters saved as CSR format at: pruned_model_csr


In [83]:
from scipy.sparse import load_npz

def load_model_from_csr(model, file_path, param_names):
    # Boş bir state_dict oluştur
    state_dict = {}

    # CSR formatındaki parametreleri yükle
    for name in param_names:
        loaded = load_npz(f"{file_path}_{name}.npz")  # Dosya adını doğru ver
        # CSR formatındaki veriyi yoğun (dense) numpy dizisine dönüştür
        dense_param = loaded.toarray()

        # Yoğun numpy dizisini torch tensörüne çevir ve modele aktar
        state_dict[name] = torch.tensor(dense_param, dtype=torch.float32)

    # Yüklenmiş state_dict'i modele aktar
    model.load_state_dict(state_dict)
    print("Model parameters loaded from CSR format.")

# Örnek kullanım
param_names = [
    'fc1.weight',
    'fc1.bias',
    'fc2.weight',
    'fc2.bias',
    'fc3.weight',
    'fc3.bias'
] # Modeldeki tüm parametre isimlerini listele
new_model = Network()
load_model_from_csr(new_model, "pruned_model_csr", param_names)

FileNotFoundError: [Errno 2] No such file or directory: 'pruned_model_csr_fc1.weight.npz'